In [5]:
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import os as os

In [6]:
def check_path(path):
    count=0
    for files in os.listdir(path):
        image_path = os.path.join(path,files)
        #print(image_path)
        count+=1
        #segmented_path=os.path.join(image_path,"NRRD")
    return count - 2

In [7]:
# (patient count* 15) check_path function will return total patient count 
folder_path = os.getcwd() + "/Incoming_Annotations"

length = check_path(folder_path)
inputs = np.zeros((length*15,256,512, 1))
outputs = np.zeros((length*15,256,512 , 9))

In [8]:
inputs.shape

(1725, 256, 512, 1)

In [9]:
def nrrd_to_np(path):
    global ann_count
    try:
        seg = sitk.GetArrayFromImage(path)
        for p in range(15):
            current_seg = seg[p]
            new_seg = np.zeros([256,512,9])
            for y,a in enumerate(current_seg):
                for x,b in enumerate(a):
                    if b>0:
                        new_seg[y][x][b-1]=1
            threshold, upper, lower = 0.5, 1, 0
            new_seg[new_seg>threshold] = upper
            new_seg[new_seg<=threshold] = lower

            #new_seg has size of 256x512x9 
            outputs[ann_count]=new_seg
            ann_count+=1
    except Exception as e:
        print(e)

In [10]:
def dcm_to_np(path):
    global dic_count
    try:
        slice_filenames = sitk.ImageSeriesReader_GetGDCMSeriesFileNames(path)
        image = sitk.ReadImage(slice_filenames)
        for x in range(15):
            current_slice = image[:, :, x]
            current_array = sitk.GetArrayFromImage(current_slice)
            inputs[dic_count]=current_array[:,:,np.newaxis]
            dic_count+=1
    except Exception as e:
        print("HAHAHA: " + str(e))
        print("But it could also be this -> " + path)

In [11]:
dic_count = 0
ann_count = 0
#set path 

for folders in sorted(os.listdir(folder_path)):
    image_path = os.path.join(folder_path, folders)
    dcm_to_np(image_path)
    print("current dic_count: " + str(dic_count))
for files in sorted(os.listdir(folder_path)):
    try:
        image_path = os.path.join(folder_path,files)
        segmented_path = os.path.join(image_path, "NRRD","")
        for entry in os.listdir(segmented_path):
            segmented_path += entry
            segmentation = sitk.ReadImage(segmented_path)
            nrrd_to_np(segmentation)
            print("current ann_count: " + str(ann_count))
    except:
        print("fuck you: " + files)

HAHAHA: Exception thrown in SimpleITK ReadImage: /tmp/SimpleITK/Code/IO/src/sitkImageSeriesReader.cxx:130:
sitk::ERROR: File names information is empty. Cannot read series.
But it could also be this -> /home/ubuntu/Incoming_Annotations/.DS_Store
current dic_count: 0
HAHAHA: Exception thrown in SimpleITK ReadImage: /tmp/SimpleITK/Code/IO/src/sitkImageSeriesReader.cxx:130:
sitk::ERROR: File names information is empty. Cannot read series.
But it could also be this -> /home/ubuntu/Incoming_Annotations/.ipynb_checkpoints
current dic_count: 0
current dic_count: 15
current dic_count: 30
current dic_count: 45
current dic_count: 60
current dic_count: 75
current dic_count: 90
current dic_count: 105
current dic_count: 120
current dic_count: 135
current dic_count: 150
current dic_count: 165
current dic_count: 180
current dic_count: 195
current dic_count: 210
current dic_count: 225
current dic_count: 240
current dic_count: 255
current dic_count: 270
current dic_count: 285
current dic_count: 300
cur

In [14]:
plt.imshow(inputs[1725].reshape(256, 512), cmap=plt.cm.bone)
plt.show()
plt.imshow(outputs[0][:, :, 8])
plt.show()

IndexError: index 1725 is out of bounds for axis 0 with size 1725

In [16]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

def unet_model(im_height, im_width, im_chan):

    input_img = Input((im_height, im_width, im_chan,), name='img')

    inp = BatchNormalization()(input_img)

    c1 = Conv2D(4, (3, 3), activation='relu', padding='same') (inp)
    a1 = MaxPooling2D((2, 2))(c1)
    c1 = Dropout(0.2)(c1)
    c1 = Conv2D(4, (3, 3), activation='relu', padding='same') (c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    cat1 = concatenate([p1, a1])

    c2 = Conv2D(8, (3, 3), activation='relu', padding='same') (cat1)
    a2 = MaxPooling2D((2, 2))(c2)
    c2 = Dropout(0.2)(c2)
    c2 = Conv2D(8, (3, 3), activation='relu', padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    cat2 = concatenate([p2, a2])

    c3 = Conv2D(16, (3, 3), activation='relu', padding='same') (cat2)
    a3 = MaxPooling2D((2, 2))(c3)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(16, (3, 3), activation='relu', padding='same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    cat3 = concatenate([p3, a3])

    c4 = Conv2D(32, (3, 3), activation='relu', padding='same') (cat3)
    a4 = MaxPooling2D((2, 2))(c4)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(32, (3, 3), activation='relu', padding='same') (c4)
    p4 = MaxPooling2D((2, 2)) (c4)

    cat4 = concatenate([p4, a4])

    c5 = Conv2D(64, (3, 3), activation='relu', padding='same') (cat4)
    a5 = MaxPooling2D((2, 2))(c5)
    c5 = Dropout(0.2)(c5)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same') (c5)
    p5 = MaxPooling2D((2, 2)) (c5)

    cat5 = concatenate([p5, a5])

    c6 = Conv2D(128, (3, 3), activation='relu', padding='same') (cat5)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same') (c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c5])
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same') (u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same') (c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c4])
    c8 = Conv2D(32, (3, 3), activation='relu', padding='same') (u8)
    c8 = Dropout(0.2)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', padding='same') (c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c3])
    c9 = Conv2D(16, (3, 3), activation='relu', padding='same') (u9)
    c9 = Dropout(0.2)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', padding='same') (c9)

    u10 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c9)
    u10 = concatenate([u10, c2])
    c10 = Conv2D(8, (3, 3), activation='relu', padding='same') (u10)
    c10 = Dropout(0.2)(c10)
    c10 = Conv2D(8, (3, 3), activation='relu', padding='same') (c10)

    u11 = Conv2DTranspose(4, (2, 2), strides=(2, 2), padding='same') (c10)
    u11 = concatenate([u11, c1], axis = 3)
    c11 = Conv2D(4, (3, 3), activation='relu', padding='same') (u11)
    c11 = Dropout(0.2)(c11)
    c11 = Conv2D(4, (3, 3), activation='relu', padding='same') (c11)

    outputs = Conv2D(9, (1, 1), activation='sigmoid') (c11)

    model = Model(inputs=[input_img], outputs=[outputs])

    return model
model = unet_model(256, 512, 1)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                (None, 256, 512, 1)  0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 512, 1)  4           img[0][0]                        
__________________________________________________________________________________________________
conv2d_23 (Conv2D)              (None, 256, 512, 4)  40          batch_normalization_1[0][0]      
__________________________________________________________________________________________________
dropout_11 (Dropout)            (None, 256, 512, 4)  0           conv2d_23[0][0]                  
__________________________________________________________________________________________________
conv2d_24 

In [17]:
from sklearn.cross_validation import train_test_split

model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["binary_crossentropy"])
results = model.fit(x=inputs, y=outputs, epochs=20, batch_size=8)

Epoch 1/20
1725/1725 [==============================] - 44s 26ms/step - loss: 1.3278 - binary_crossentropy: 0.6752
Epoch 2/20
1725/1725 [==============================] - 35s 20ms/step - loss: 1.0132 - binary_crossentropy: 0.5477
Epoch 3/20
1725/1725 [==============================] - 35s 20ms/step - loss: 0.9047 - binary_crossentropy: 0.4920
Epoch 4/20
1725/1725 [==============================] - 34s 20ms/step - loss: 0.8471 - binary_crossentropy: 0.4618
Epoch 5/20
1725/1725 [==============================] - 36s 21ms/step - loss: 0.7634 - binary_crossentropy: 0.3602
Epoch 6/20
1725/1725 [==============================] - 36s 21ms/step - loss: 0.6710 - binary_crossentropy: 0.2943
Epoch 7/20
1624/1725 [===========================>..] - ETA: 2s - loss: 0.6286 - binary_crossentropy: 0.2164

KeyboardInterrupt: 